# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    # print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            # print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [5]:
# sample df from event_datafile_new.csv
df = pd.read_csv('event_datafile_new.csv')
df.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,A Fine Frenzy,Anabelle,F,0,Simpson,267.91138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",256,Almost Lover (Album Version),69
1,Nirvana,Aleena,F,0,Kirby,214.77832,paid,"Waterloo-Cedar Falls, IA",237,Serve The Servants,44
2,Television,Aleena,F,1,Kirby,238.49751,paid,"Waterloo-Cedar Falls, IA",237,See No Evil (Remastered LP Version),44
3,JOHN COLTRANE,Aleena,F,2,Kirby,346.43546,paid,"Waterloo-Cedar Falls, IA",237,Blues To Bechet (LP Version),44
4,NOFX,Aleena,F,3,Kirby,80.79628,paid,"Waterloo-Cedar Falls, IA",237,It's My Job To Keep Punk Rock Elite,44


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [7]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [9]:
# for mapping column name to index
col_index_dict = {v: i for i, v in enumerate(df.columns)}
print(col_index_dict)

{'artist': 0, 'firstName': 1, 'gender': 2, 'itemInSession': 3, 'lastName': 4, 'length': 5, 'level': 6, 'location': 7, 'sessionId': 8, 'song': 9, 'userId': 10}


## Query 1:  
Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [10]:
# check unique column combination on df
df.drop_duplicates(['sessionId','itemInSession']).shape
# add sessionId, itemInSession two field can make sure each record is unique.

(6820, 11)

In [11]:
query = """
    CREATE TABLE IF NOT EXISTS song_library (
        artist text,
        song text,
        length float,
        sessionId int,
        itemInSession int,
        PRIMARY KEY (sessionId, itemInSession)
    )
"""
try:
    session.execute(query)
except Exception as e:
    print(e)                

In [12]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_library (artist, song, length, sessionId, itemInSession)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        try:
            session.execute(
                query, (
                    line[col_index_dict['artist']],
                    line[col_index_dict['song']],
                    float(line[col_index_dict['length']]),
                    int(line[col_index_dict['sessionId']]),
                    int(line[col_index_dict['itemInSession']])
                )
            )
        except Exception as e:
            print(e)
            break

#### Do a SELECT to verify that the data have been inserted into each table

In [13]:
query = "select * from song_library WHERE sessionId=338 and itemInSession=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
q_df = pd.DataFrame([{'artist': row.artist, 'song': row.song, 'length': row.length} for row in rows])
q_df[['artist', 'song', 'length']]

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


### description of the query 1
query 1 need to use two field for where condition, so it need primary key on sessionId and itemInSession

## Query 2:
Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) 
for userid = 10, sessionid = 182

In [14]:
# check unique column combination on df
len(df.drop_duplicates(['userId', 'sessionId', 'itemInSession']))
# Because userId is the first where condition, so it must be partition key.
# Then, add sessionId, itemInSession two clustered keys to make sure each record is unique.
# SessionId also is the where condition, it's order need to in front of itemInSession.

6820

In [15]:
query = """
    CREATE TABLE IF NOT EXISTS song_library_users (
        artist text,
        song text,
        firstName text,
        lastName text,
        sessionId int,
        itemInSession int,
        userId int,
        PRIMARY KEY (userId, sessionId, itemInSession)
    )
"""
try:
    session.execute(query)
except Exception as e:
    print(e)        

In [16]:
# insert data for Query 2
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_library_users (artist, song, firstName, lastName, sessionId, itemInSession, userId)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        try:
            session.execute(
                query, (
                    line[col_index_dict['artist']],
                    line[col_index_dict['song']],
                    line[col_index_dict['firstName']],
                    line[col_index_dict['lastName']],
                    int(line[col_index_dict['sessionId']]),
                    int(line[col_index_dict['itemInSession']]),
                    int(line[col_index_dict['userId']])
                )
            )
        except Exception as e:
            print(e)
            break

In [17]:
## execute Query 2
query = """
    SELECT artist, song, firstName, lastName 
    FROM song_library_users
    WHERE userid=10 AND sessionid=182
"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

q_df = pd.DataFrame([{
    'artist': row.artist,
    'song': row.song,
    'firstName': row.firstname,
    'lastName': row.lastname,
} for row in rows])
q_df[['artist', 'song', 'firstName', 'lastName']]

,artist,song,firstName,lastName
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


### description of the query 2
query 2 need song sorted by itemInSession, so itemInSession must be the clustered key.
userId is the first where condition, so it must be partition key.
Then, sessionId is in the where condition, and itemInSession is not.
So, sessionId is clustered key too, and it's order must be in front of itemInSession clustered key.

## Query 3: 
Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [37]:
# check unique column combination on df
len(df.drop_duplicates(['song', 'userId', 'sessionId', 'itemInSession']))
# Because users may have same name, so we need userId for primary key.
# and add sessionId, itemInSession to make sure each record is unique.

6820

In [38]:
query = """
    CREATE TABLE IF NOT EXISTS users_query_by_song (
        song text,
        userId int,
        firstName text,
        lastName text,
        sessionId int,
        itemInSession int,
        PRIMARY KEY (song, userId, sessionId, itemInSession)
    )
"""
try:
    session.execute(query)
except Exception as e:
    print(e)                            

In [39]:
# insert data for Query 3
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users_query_by_song (song, userId, firstName, lastName, sessionId, itemInSession)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s)"
        try:
            session.execute(
                query, (
                    line[col_index_dict['song']],
                    int(line[col_index_dict['userId']]),
                    line[col_index_dict['firstName']],
                    line[col_index_dict['lastName']],
                    int(line[col_index_dict['sessionId']]),
                    int(line[col_index_dict['itemInSession']])
                )
            )
        except Exception as e:
            print(e)
            break

In [40]:
## execute Query 3
query = """
    SELECT firstName, lastName 
    FROM users_query_by_song
    WHERE song='All Hands Against His Own'
"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

q_df = pd.DataFrame([{
    'firstName': row.firstname,
    'lastName': row.lastname,
} for row in rows])
q_df[['firstName', 'lastName']]

,firstName,lastName
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### description of the query 3
query 3 need every user name (first and last) in the music app history who listened to the given song name.
Because users may have same name, so we need userId for primary key to make each user is unique.
and add sessionId, itemInSession to make sure each record is unique.

### Drop the tables before closing out the sessions

In [41]:
# drop table for query 1
try:
    session.execute("DROP TABLE IF EXISTS song_library")
except Exception as e:
    print(e)

# drop table for query 2
try:
    session.execute("DROP TABLE IF EXISTS song_library_users")
except Exception as e:
    print(e)

# drop table for query 3
try:
    session.execute("DROP TABLE IF EXISTS users_query_by_song")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [42]:
session.shutdown()
cluster.shutdown()